# Generate input data for exercise

The data is a simple wavelength spectrum with some peaks and and affine background stored as a Scipp HDF5 file.

In [ ]:
import scipp as sc
import numpy as np
import plopp as pp

pp.patch_scipp()
%matplotlib widget

# Prepare data

In [ ]:
rng = np.random.default_rng(74712)

In [ ]:
x = sc.linspace("wavelength", 0.3, 1.2, 500, unit="angstrom")

offset = sc.scalar(0.2, unit="counts")
slope = sc.scalar(-0.1, unit="counts/angstrom")
background = offset + slope * x

mu0 = sc.scalar(0.42, unit="angstrom")
sig0 = sc.scalar(0.1, unit="angstrom")
peak0 = sc.exp(-((x - mu0) ** 2) / sig0**2) / 2 / np.sqrt(2 * np.pi)

mu1 = sc.scalar(0.65, unit="angstrom")
sig1 = sc.scalar(0.12, unit="angstrom")
peak1 = sc.exp(-((x - mu1) ** 2) / sig1**2) / 2 / np.sqrt(2 * np.pi) / 1.5

mu2 = sc.scalar(0.89, unit="angstrom")
sig2 = sc.scalar(0.06, unit="angstrom")
peak2 = sc.exp(-((x - mu2) ** 2) / sig2**2) / 2 / np.sqrt(2 * np.pi)

peaks = peak0 + peak1 + peak2
peaks.unit = background.unit

noise = sc.array(
    dims=x.dims, values=rng.normal(0.0, 0.005, background.shape), unit=background.unit
)

lineshape = (background + peaks + noise) * 10_000
da = sc.DataArray(lineshape, coords={"wavelength": x})

# Workflow

`da` would be loaded from a file obtained via Scitacean.

In [ ]:
da.plot(ls="-", marker=None)

Get from SciCat

In [ ]:
proton_charge = sc.scalar(900, unit="uAh")

Find background.
There are multiple ways to do this, esp. since the background is not constant.

In [ ]:
bg = sc.min(da)

In [ ]:
corrected = (da - background) / proton_charge

In [ ]:
corrected.plot(ls="-", marker=None)

Save `corrected` to file and make SciCat dataset.

# Save data

In [ ]:
da.to_hdf5("spectrum-raw.h5")